## Handling RAM-like memory

In [1]:
MEMSIZE = 1000 # total size of memory
mem = [None for _ in range(MEMSIZE)] # memory
stack_pos = 0 # head to first free position in stack
heap_pos = MEMSIZE # head to last occupied position in heap

In [2]:
""" init mem """
def mem_init():
    global mem, stack_pos, heap_pos
    mem = [None for _ in range(MEMSIZE)] # memory
    stack_pos = 0 # head to first free position in stack
    heap_pos = MEMSIZE # head to last occupied position in heap
     
""" print contents of mem """
def mem_dump(sigma):
    print("sigma = ", sigma, " mem = ", 
          [ mem[i] for i in range(stack_pos)] + ["..."] + 
           [ mem[i] for i in range(heap_pos, MEMSIZE)])

""" heap allocation and release """
def malloc(npositions):
    global heap_pos
    heap_pos -= npositions
    return heap_pos

def free(address):
    pass # no action here, garbage collection is a complex task

## Expression and Boolean Parser Revised

In [3]:
from lark import Lark, InlineTransformer 


""" gexp is boolexp grammar on top of exp grammar: 
    numeric operators have higher precedence than boolean operators.
"""
grammar_gexp = """
    ?start: boolexp
    ?boolexp: booland
        | boolexp "or" booland -> orop
    ?booland: boolnot
        | booland "and" boolnot -> andop
    ?boolnot: boolcmp
        | "not" boolnot     -> notop
    ?boolcmp: expcmp
        | boolcmp "==" expcmp -> eq
    ?expcmp: exp
        | expcmp "<=" exp      -> lte
        | expcmp "<" exp       -> lt
    ?exp: product
        | exp "+" product   -> add
        | exp "-" product   -> sub
    ?product: atom
        | product "*" atom  -> mul
        | product "/" atom  -> div
    ?atom: NUMBER           -> number
         | "-" atom         -> neg
         | "True"           -> true
         | "False"          -> false
         | NAME             -> var
         | "(" boolexp ")"
    %import common.CNAME -> NAME
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

class Tree_GExp(InlineTransformer):
    number = float

    def var(self, name):
        return ['variable', str(name)]
    
    def false(self):
        return ['bool_const', False]
    
    def true(bool_const):
        return ['bool_const', True]
    
    def orop(self, left, right):
        return ['or', left, right]
    
    def andop(self, left, right):
        return ['and', left, right]

    def notop(self, value):
        return ['not', value]

    def lte(self, left, right):
        return ['<=', left, right]

    def lt(self, left, right):
        return ['<', left, right]

    def eq(self, left, right):
        return ['==', left, right]

    def number(self, value):
        return ['number_const', float(value)]

    def add(self, left, right):
        return ['+', left, right]
    
    def sub(self, left, right):
        return ['-', left, right]
    
    def mul(self, left, right):
        return ['*', left, right]
    
    def div(self, left, right):
        return ['/', left, right]
    
    def neg(self, left):
        return ['-', left]

def solve_exp(exp, sigma):
    op = exp[0]
    """ Algebra of numbers """
    if op=='number_const':
        return exp[1]
    if op=='variable':
        return sigma[exp[1]]
    if op=='+':
        return solve_exp(exp[1], sigma) + solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==3:
        return solve_exp(exp[1], sigma) - solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==2:
        return -solve_exp(exp[1], sigma)
    if op=='*':
        return solve_exp(exp[1], sigma) * solve_exp(exp[2], sigma)
    if op=='/':
        return solve_exp(exp[1], sigma) / solve_exp(exp[2], sigma)
    """ Algebra of Booleans """
    if op=='bool_const':
        return exp[1]
    if op=='or':
        return bool(solve_exp(exp[1], sigma)) or bool(solve_exp(exp[2], sigma))
    if op=='and':
        return bool(solve_exp(exp[1], sigma)) and bool(solve_exp(exp[2], sigma))
    if op=='not':
        return not solve_exp(exp[1], sigma)
    if op=='==':
        return solve_exp(exp[1], sigma) == solve_exp(exp[2], sigma)
    if op=='<=':
        return solve_exp(exp[1], sigma) <= solve_exp(exp[2], sigma)
    if op=='<':
        return solve_exp(exp[1], sigma) < solve_exp(exp[2], sigma)  

def get_parser_gexp():
    parser = Lark(grammar_gexp, parser='lalr', transformer=Tree_GExp())
    return parser.parse

def merge_two_dicts(d1, d2):
    d = d1.copy()
    d.update(d2)
    return d

In [4]:
if __name__ == '__main__':
    parse_exp = get_parser_gexp()
    
    E1 = parse_exp("x + 2*y")
    print(E1)
    
    sigma = { 'x':0, 'y':1 }
    mem[0] = 2
    mem[1] = 3
    stack_pos = 2
    mem_dump(sigma)
    print('E1 = ', solve_exp(E1, sigma))

    """ new address associated to x """
    sigma = { 'x':2, 'y':1 }
    mem[2] = 1
    stack_pos += 1
    mem_dump(sigma)
    print('E1 = ', solve_exp(E1, sigma))

    """ variable may share same address """
    sigma = { 'x':0, 'y':0 }
    mem[0] = 2
    stack_pos = 1
    mem_dump(sigma)
    print('E1 = ', solve_exp(E1, sigma))

['+', ['variable', 'x'], ['*', ['number_const', 2.0], ['variable', 'y']]]
sigma =  {'x': 0, 'y': 1}  mem =  [2, 3, '...']
E1 =  2.0
sigma =  {'x': 2, 'y': 1}  mem =  [2, 3, 1, '...']
E1 =  4.0
sigma =  {'x': 0, 'y': 0}  mem =  [2, '...']
E1 =  0.0


## Parser

In [5]:
# -*- coding: utf-8 -*-

""" start anaconda/prompt in administration mode
    > pip install lark-parser """
from lark import Lark 

grammar_cmd = """
    ?start: cmd
    ?cmd: cmdbase
        | cmd cmdbase           -> seq
    ?cmdbase: "var" NAME ";"    -> vardecl
        | NAME "=" gexp ";"     -> assignment
        | "*" NAME "=" gexp ";" -> derefassignment
        | "{" cmd "}"       -> block
        | "if" "(" gexp ")" cmdbase "else" cmdbase -> ifelse
        | "if" "(" gexp ")" cmdbase     -> ifcond
        | "while" "(" gexp ")" cmdbase  -> loop
        | "print" "(" gexp ")" ";"      -> pprint
        | "free" "(" NAME ")" ";"       -> free
""" + grammar_gexp.replace("\n    ?start:", "    ?gexp:").\
replace("\n         | NAME", """
         | "&" NAME           -> ref
         | "*" NAME           -> deref
         | "malloc" "(" gexp ")"   -> malloc
         | NAME "(" gexp ")"  -> call
         | NAME""")

class Tree_Cmd(Tree_GExp):
    
    def vardecl(self, var):
        return ['decl', str(var)]

    def assignment(self, var, expr):
        return ['=', str(var), expr]

    def derefassignment(self, var, expr):
        return ['deref=', str(var), expr]

    def seq(self, cmd1, cmd2):
        if cmd1[0]=='seq':
            return ['seq'] + cmd1[1:] + [cmd2]
        return ['seq', cmd1, cmd2]

    def block(self, cmd):
        return ['block', cmd]

    def ifelse(self, cond, cmdif, cmdelse):
        return ['if', cond, cmdif, cmdelse]
    
    def ifcond(self, cond, cmd):
        return ['if', cond, cmd]

    def loop(self, cond, cmd):
        return ['while', cond, cmd]

    def pprint(self, gexp):
        return ['print', gexp]

    def ref(self, var):
        return ['ref', str(var)]

    def deref(self, var):
        return ['deref', str(var)]

    def malloc(self, exp):
        return ['malloc', exp]

    def free(self, var):
        return ['free', str(var)]

def get_parser_cmd():
    parser = Lark(grammar_cmd, parser='lalr', transformer=Tree_Cmd())
    return parser.parse 

""" test """
if __name__ == '__main__':
    print(grammar_cmd)
    parse = get_parser_cmd()
    
    C1 = parse("var x; x = 2; var y; y = 1; var z; z = x+2*y; print(z);")
    print(C1)
        


    ?start: cmd
    ?cmd: cmdbase
        | cmd cmdbase           -> seq
    ?cmdbase: "var" NAME ";"    -> vardecl
        | NAME "=" gexp ";"     -> assignment
        | "*" NAME "=" gexp ";" -> derefassignment
        | "{" cmd "}"       -> block
        | "if" "(" gexp ")" cmdbase "else" cmdbase -> ifelse
        | "if" "(" gexp ")" cmdbase     -> ifcond
        | "while" "(" gexp ")" cmdbase  -> loop
        | "print" "(" gexp ")" ";"      -> pprint
        | "free" "(" NAME ")" ";"       -> free
    ?gexp: boolexp
    ?boolexp: booland
        | boolexp "or" booland -> orop
    ?booland: boolnot
        | booland "and" boolnot -> andop
    ?boolnot: boolcmp
        | "not" boolnot     -> notop
    ?boolcmp: expcmp
        | boolcmp "==" expcmp -> eq
    ?expcmp: exp
        | expcmp "<=" exp      -> lte
        | expcmp "<" exp       -> lt
    ?exp: product
        | exp "+" product   -> add
        | exp "-" product   -> sub
    ?product: atom
        | product "*" atom  -> mul

## Interpreter

In [6]:
def solve_exp(exp, sigma):
    op = exp[0]
    """ Algebra of numbers """
    if op=='number_const':
        return exp[1]
    if op=='variable':
        return mem[sigma[exp[1]]] # changed wrt interpreter3.py
    if op=='+':
        return solve_exp(exp[1], sigma) + solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==3:
        return solve_exp(exp[1], sigma) - solve_exp(exp[2], sigma)
    if op=='-' and len(exp)==2:
        return -solve_exp(exp[1], sigma)
    if op=='*':
        return solve_exp(exp[1], sigma) * solve_exp(exp[2], sigma)
    if op=='/':
        return solve_exp(exp[1], sigma) / solve_exp(exp[2], sigma)
    """ Algebra of Booleans """
    if op=='bool_const':
        return exp[1]
    if op=='or':
        return bool(solve_exp(exp[1], sigma)) or bool(solve_exp(exp[2], sigma))
    if op=='and':
        return bool(solve_exp(exp[1], sigma)) and bool(solve_exp(exp[2], sigma))
    if op=='not':
        return not solve_exp(exp[1], sigma)
    if op=='==':
        return solve_exp(exp[1], sigma) == solve_exp(exp[2], sigma)
    if op=='<=':
        return solve_exp(exp[1], sigma) <= solve_exp(exp[2], sigma)
    if op=='<':
        return solve_exp(exp[1], sigma) < solve_exp(exp[2], sigma)
    # new operators wrt interpreter3.py here
    if op=='ref':
        return sigma[exp[1]]
    if op=='deref':
        return mem[mem[sigma[exp[1]]]]
    if op=='malloc':
        return malloc(int(solve_exp(exp[1], sigma)))

def merge_two_dicts(d1, d2):
    d = d1.copy()
    d.update(d2)
    return d
   
def solve_cmd(cmd, sigma):
    global stack_pos
    op = cmd[0]
    if op=='decl':
        var = cmd[1]
        """ stack pushing """
        address = stack_pos
        stack_pos += 1
        return merge_two_dicts(sigma, {var:address})
    if op=='=':
        var = cmd[1]
        expr = cmd[2]
        mem[sigma[var]] = solve_exp(expr, sigma)
        return sigma
    if op=='seq':
        for c in cmd[1:]:
            sigma = solve_cmd(c, sigma)
        return sigma
    if op=='print':
        val = solve_exp(cmd[1], sigma)
        print(val)
        return sigma
    if op=='block':
        tmp = stack_pos
        solve_cmd(cmd[1], sigma)
        """ stack popping """
        stack_pos = tmp
        return sigma
    if op=='if':
        b = solve_exp(cmd[1], sigma)
        if b:
            return solve_cmd(cmd[2], sigma)
        if len(cmd)==4: #if else
            return solve_cmd(cmd[3], sigma)
        return sigma
    if op=='while':
        b = solve_exp(cmd[1], sigma)
        if not b:
            return sigma
        body = cmd[2]
        sigma = solve_cmd(body, sigma)
        return solve_cmd(cmd, sigma)
    if op=='deref=':
        var = cmd[1]
        expr = cmd[2]
        mem[int(mem[sigma[var]])] = solve_exp(expr, sigma)
        return sigma
    if op=='free':
        var = cmd[1]
        free(mem[sigma[var]])
        mem[sigma[var]] = None
        return sigma

In [7]:
parse = get_parser_cmd()
    
c_code = """ C-code to run at http://pythontutor.com/c.html#mode=edit
float x;
x = 2;
float y;
y = 1;
float z;
z = x + 2*y;
"""

C1 = parse("var x; x = 2; var y; y = 1; var z; z = x+2*y; print(z);")
print("-- C1 --")
#print(C1)  
sigma = {}
mem_init()
sigmares = solve_cmd(C1, sigma)
mem_dump(sigmares)

-- C1 --
4.0
sigma =  {'x': 0, 'y': 1, 'z': 2}  mem =  [2.0, 1.0, 4.0, '...']


In [8]:
C2 = parse("{ var x; x = 2; var y; y = 1; var z; z = x+2*y; print(z);}")
print("-- C2 --")
#print(C2)
mem_init()
sigmares = solve_cmd(C2, sigma)
mem_dump(sigmares)

c_code = """ C-code
float x;
x = 2;
{
    float y;
    y = 1;
    float z;
    z = 2*y;
}
"""

-- C2 --
4.0
sigma =  {}  mem =  ['...']


In [9]:
C3 = parse("{ var x; x = 2; {var y; y = 1; var z; z = 2*y; print(z);} }")
print("-- C3 --")
#print(C3)
mem_init()
sigmares = solve_cmd(C3, sigma)
mem_dump(sigmares)

""" C-code
int i;
i = 5;
while(0<i)
{
    printf("%d\n", i);
    i--;
}
"""

-- C3 --
2.0
sigma =  {}  mem =  ['...']


' C-code\nint i;\ni = 5;\nwhile(0<i)\n{\n    printf("%d\n", i);\n    i--;\n}\n'

In [10]:
C4 = parse("var i; i=5; while(0<i) { print(i); i = i-1; }")
print("-- C4 --")
#print(C4)
mem_init()
sigmares = solve_cmd(C4, sigma)
mem_dump(sigmares)

c_code = """ C-code
float x;
x = 3;
float *y;
y = &x;
*y = *y + 1;
printf("%f\n", x);
"""

-- C4 --
5.0
4.0
3.0
2.0
1.0
sigma =  {'i': 0}  mem =  [0.0, '...']


In [11]:
C5 = parse("var x; x=3; var y; y = &x; *y = *y + 1; print(x);")
print("-- C5 --")
#print(C5)
mem_init()
sigmares = solve_cmd(C5, sigma)
mem_dump(sigmares)

c_code = """ C-code
float x;
x = 3;
float *y;
y = malloc(sizeof(float));
*y = x + 1;
printf("%f %p\n", *y, y);
free(y);
"""

-- C5 --
4.0
sigma =  {'x': 0, 'y': 1}  mem =  [4.0, 0, '...']


In [12]:
C6 = parse("var x; x=3; var y; y = malloc(1); *y = x + 1; print(*y); print(y); ")
print("-- C6 --")
#print(C6)
mem_init()
sigmares = solve_cmd(C6, sigma)
mem_dump(sigmares)

c_code = """ C-code
int i;
i = 5;
float *y;
y = malloc(sizeof(float)*i);
float x;
x=5;
while(0<i)
{
    i--;
    *(y+i) = x;
    x--;
}
"""

-- C6 --
4.0
999
sigma =  {'x': 0, 'y': 1}  mem =  [3.0, 999, '...', 4.0]


In [13]:
C7 = parse("""
    var i; 
    i=5; 
    var y; 
    y = malloc(i); 
    var x; 
    x = 5; 
    while(0<i) 
    { 
        i = i-1;
        var z;
        z = y+i;
        *z = x;
        x = x-1;
    }""".replace("\n",""))

print("-- C7 --")
#print(C7)
mem_init()
sigmares = solve_cmd(C7, sigma)
mem_dump(sigmares)

-- C7 --
sigma =  {'i': 0, 'y': 1, 'x': 2}  mem =  [0.0, 995, 0.0, '...', 1.0, 2.0, 3.0, 4.0, 5.0]
